In [77]:
import pandas as pd
import numpy as np

In [78]:
test = pd.read_csv('../__idao_data/tr1/test.csv', index_col="id")
sub = pd.read_csv('../__idao_data/tr1/submission.csv', index_col="id")

In [79]:
ans_team = pd.read_csv('submission.csv', index_col="id")
ans_true = pd.read_csv('../__idao_ans/tr1/ans.csv', index_col="id")

In [80]:
print(test.shape)
print(sub.shape)
print(ans_team.shape)
print(ans_true.shape)


(284071, 8)
(284071, 6)
(284071, 6)
(284071, 6)


In [81]:
test.head()

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,
3927,1,2014-02-01T00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
3928,1,2014-02-01T00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
3929,1,2014-02-01T00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3930,1,2014-02-01T01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
3931,1,2014-02-01T01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [82]:
sub.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
3928,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
3929,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3930,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
3931,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [83]:
ans_team.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-24789.953903,-10909.104984,6566.102066,2.614835,-1.304002,1.087387
3928,-21084.919391,-12382.982109,7835.944162,3.219872,-0.994565,0.896041
3929,-16576.617680,-13377.863994,8802.253404,3.878736,-0.539572,0.601988
3930,-11198.146103,-13654.191287,9294.803223,4.583997,0.157488,0.134757
3931,-4928.171451,-12793.704838,9011.342391,5.252874,1.294504,-0.653884


In [84]:
ans_true.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-24791.149338,-10910.791621,6570.495423,2.614232,-1.303894,1.087104
3928,-21086.983936,-12384.575495,7839.995470,3.219101,-0.994535,0.895794
3929,-16579.792530,-13379.503655,8806.047260,3.877753,-0.539691,0.601842
3930,-11202.738191,-13656.146227,9298.539686,4.582746,0.157074,0.134846
3931,-4934.257986,-12796.478611,9015.246708,5.251415,1.293472,-0.653239


In [85]:
ans_true.loc[3927]

x    -24791.149338
y    -10910.791621
z      6570.495423
Vx        2.614232
Vy       -1.303894
Vz        1.087104
Name: 3927, dtype: float64

# Метрики для всех данных

In [86]:
def smape(pred, ans):
    smape = np.mean(np.abs(pred - ans) / (np.abs(pred) + np.abs(ans))) * 100
#     return 100 - smape
    return smape

In [89]:
from numpy.linalg import norm
def smape_new_vector_norm(pred, ans, av_score=True):
    scores = []
    if not av_score:
        r_losses = v_losses = []
    for sat_id in pd.unique(test["sat_id"]):
        idxs = (test["sat_id"] == sat_id).values
        p = pred[idxs]
        a = ans[idxs]
        loss = p - a
        # coordinates
        r_loss = norm(loss[:, :3], axis=1) / (norm(p[:, :3], axis=1) + norm(a[:, :3], axis=1))
        r_loss = np.mean(r_loss)
        # velocities
        v_loss = norm(loss[:, 3:], axis=1) / (norm(p[:, 3:], axis=1) + norm(a[:, 3:], axis=1))     
        v_loss = np.mean(v_loss)
        # score
        scores.append(r_loss + v_loss)
        if not av_score:
            r_losses.append(r_loss)            
            v_losses.append(v_loss)
    if av_score:
        return np.mean(scores)
    else:
        return np.array(scores), np.array(r_losses), np.array(v_losses)

In [90]:
smape_team = smape(ans_team.values, ans_true.values)
mse_team = np.square(ans_team.values - ans_true.values).mean()
rmse_team = np.square(ans_team.values - ans_true.values).mean() ** 0.5
new_smape_team = smape_new_vector_norm(ans_team.values, ans_true.values)

print("Team")
print(f"SMAPE: {smape_team}")
print(f"MSE: {mse_team}")
print(f"RMSE (СКО): {rmse_team}")
print(f"new SMAPE: {new_smape_team}")

smape_sub = smape(sub.values, ans_true.values)
mse_sub = np.square(sub.values - ans_true.values).mean()
rmse_sub = np.square(sub.values - ans_true.values).mean() ** 0.5
new_smape_sub = smape_new_vector_norm(sub.values, ans_true.values)

print("\nDefault (SGP4)")
print(f"SMAPE: {smape_sub}")
print(f"MSE: {mse_sub}")
print(f"RMSE (СКО): {rmse_sub}")
print(f"new SMAPE: {new_smape_sub}")

def improvement(a, b):
    return (a - b) / a * 100

print("\nImprovement")
print(f"SMAPE: {improvement(smape_sub, smape_team):.5}%")
print(f"MSE: {improvement(mse_sub, mse_team):.5}%")
print(f"RMSE (СКО): {improvement(rmse_sub, rmse_team):.5}%")
print(f"new SMAPE: {improvement(new_smape_sub, new_smape_team):.5}%")

Team
SMAPE: 10.281815562668708
MSE: 649353031.9602685
RMSE (СКО): 25482.40632201497
new SMAPE: 0.08774738523961789

Default (SGP4)
SMAPE: 34.890318581695176
MSE: 110471199.08459651
RMSE (СКО): 10510.528011693634
new SMAPE: 0.4874228984024747

Improvement
SMAPE: 70.531%
MSE: -487.8%
RMSE (СКО): -142.45%
new SMAPE: 81.998%


In [1]:
def improvement(a, b):
    return (a - b) / a * 100

In [94]:
((sub.values - ans_true.values) ** 2).mean() ** 0.5

10510.528011693634

In [2]:
improvement(66, 96)

-45.45454545454545

In [61]:
print(f"SMAPE imp.: {smape(sub.values, ans_true.values) / smape(ans_team.values, ans_true.values)*100:.5}%")
print(f"RMSE (СКО) imp.: {(np.square(sub.values - ans_true.values).mean() ** 0.5 / np.square(ans_team.values - ans_true.values).mean() ** 0.5)*100:.5}%")
print(f"new SMAPE imp.: {smape_new_vector_norm(sub.values, ans_true.values)}")

SMAPE imp.: 339.34%
RMSE (СКО) imp.: 41.246%
new SMAPE imp.: 0.4874228984024747


In [ ]:
test_data_full.drop(['epoch', 'sat_id'], axis=1)

In [35]:
a = test_data_full.drop(['epoch', 'sat_id'], axis=1).values
b = submission_true.values

In [36]:
np.abs(a - b) / (np.abs(a) + np.abs(b))

array([[1.        , 1.        , 1.        , 0.14277411, 0.34391936,
        0.79990897],
       [1.        , 1.        , 1.        , 0.30092395, 1.        ,
        0.5214157 ],
       [1.        , 1.        , 0.9379371 , 0.49093286, 1.        ,
        1.        ],
       ...,
       [       nan,        nan,        nan,        nan,        nan,
               nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan]])

In [37]:
np.isnan(a).sum()

0

In [38]:
np.isnan(b).sum()

1484310

In [39]:
b

array([[ 1.14394917e+04,  3.36862633e+04, -4.31864618e+02,
         3.25091251e+00, -1.41906149e+00,  9.01315980e-02],
       [ 2.16829472e+04,  2.72107878e+04, -1.09041525e+02,
         2.72844375e+00, -2.37517892e+00,  9.85853068e-02],
       [ 2.95863807e+04,  1.78147118e+04,  2.25258953e+02,
         1.88445483e+00, -3.11537702e+00,  9.66560640e-02],
       ...,
       [            nan,             nan,             nan,
                    nan,             nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan]])